
### 1. **Objetivos**

**Objetivo do Modelo**:
- O objetivo deste modelo é prever dois fatores críticos para a produção:
  
  1. **Total de defeitos** esperados para uma nova ordem de produção, com base em medidas físicas (peso, comprimento e diâmetro).
  
  2. **Dias críticos de defeitos**, ou seja, prever em qual dia da produção será mais provável ocorrer o maior número de defeitos.

Essas previsões serão usadas para:
- Melhorar a **qualidade da produção**, permitindo ações preventivas em dias críticos.
- **Otimizar o processo de produção**, com base no histórico de ordens anteriores.
- **Ajudar o time de vendas e orçamento** a planejar e dimensionar ordens de forma mais eficiente, antecipando o comportamento de defeitos com base nas características do pedido e na duração da ordem.

### 2. **Solução**

**Solução Proposta**:
- Para prever o **total de defeitos**, usaremos um modelo de regressão, que será treinado com dados históricos sobre medidas físicas (peso, comprimento e diâmetro) e o total de defeitos observados em ordens anteriores.
- Para prever os **dias críticos de defeitos**, utilizaremos uma análise de sazonalidade, que considera o padrão de defeitos ao longo dos dias de produção de cada ordem. Isso nos ajudará a identificar quais dias (por exemplo, dia 2) têm maior chance de apresentar defeitos.
- Além disso, a **duração da ordem** (quantidade de dias de produção) será um parâmetro importante para ajustar as previsões. Ordens mais longas tendem a ter mais defeitos, e a duração influencia o comportamento dos defeitos ao longo dos dias.

**Modelo Utilizado**:
- **Regressão (ex: Gradient Boosting)**: Para prever o **total de defeitos** com base nas características do pedido e a duração da ordem.
- **Análise de série temporal**: Para identificar **quais dias** terão maior probabilidade de defeitos ao longo da produção.

### 3. **Como o Modelo Vai Funcionar (Inputs e Outputs)**

#### Exemplo:
**Inputs (Entradas do Modelo)**:
- **Medidas da Ordem**:
  - Peso médio (g)
  - Comprimento médio (mm)
  - Diâmetro
- **Duração da Ordem**:
  - Número de dias que a produção vai durar (fornecido pelo time de vendas ou orçamento).
- **Dados Históricos**: O modelo é treinado com ordens passadas, suas medidas e os defeitos observados.

**Processamento**:
- O modelo será treinado com essas entradas para aprender os padrões que relacionam as **medidas físicas** e a **duração** com os **defeitos** observados em ordens passadas.
- O modelo ajustará as previsões com base nos padrões de **sazonalidade** dos defeitos, observando quais dias em ordens passadas têm mais defeitos.

**Outputs (Saídas do Modelo)**:
- **Previsão do Total de Defeitos**: Para uma nova ordem, o modelo preverá o total de defeitos, com base nas novas medidas fornecidas.
- **Previsão dos Dias Críticos de Defeitos**: O modelo também preverá os dias críticos de defeitos, ou seja, os dias específicos em que a ordem pode apresentar mais defeitos, com base no histórico de ordens anteriores e sua sazonalidade.

#### Exemplo de como isso pode ser explicado em termos simples:
- **Input**: Você fornecerá ao modelo as **medidas físicas** de uma nova ordem (peso, comprimento, diâmetro) e o **número de dias de produção** (duração).
- **Output**: O modelo retornará dois tipos de informações:
  
  1. **O total de defeitos esperados** para essa ordem.
  
  2. **Quais dias da ordem** terão maior probabilidade de defeitos, ajudando a identificar **o dia crítico** de defeitos.

### Exemplo de Diagrama ou Fluxograma:
Você também pode usar um fluxograma simples para ilustrar o processo de previsão do modelo, como por exemplo:

1. **Input**: Medidas da ordem (peso, comprimento, diâmetro, duração da ordem)

2. **Processamento**:
   - Treinamento com dados históricos (medidas + defeitos + duração + sazonalidade).
   - Ajuste de previsões com base na duração e no padrão de defeitos.

3. **Output**:
   - Previsão do total de defeitos.
   - Previsão dos dias críticos de defeitos.

### 4. **Conclusão**

- **Benefícios**:
  
  - **Previsões mais precisas** de defeitos, com base em dados reais e a duração de cada ordem.
  
  - **Ação preventiva**: Saber antecipadamente quais dias têm mais chance de defeitos pode otimizar os processos de produção.
  
  - **Apoio à decisão**: Ajudar os times de produção e vendas a planejar melhor as ordens e a controlar a qualidade da produção.

## Implementação do Modelo

### Importando bibliotecas

In [513]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

### Importando dataset

In [514]:
dados = pd.read_csv('../dados/medidas_geral_e_defeitos.csv')
df = pd.DataFrame(dados)

In [515]:
df = df.set_index(['Ordem de prod', 'Ref. do Artigo', 'Data de prod'])

In [516]:
df

Temperatura (F°)  Peso médio (g)  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03             2126.97           169.3   
                             2024-01-04             2118.92           169.7   
                             2024-01-05             2125.00           173.6   
                             2024-01-06             2125.40           170.2   
                             2024-01-07             2126.76           171.7   
...                                                     ...             ...   
194121        C0516.0000R    2024-06-26             2154.44           169.0   
                             2024-06-27             2152.32           168.9   
                             2024-06-28             2145.60           169.3   
                             2024-06-29             2145.60           168.8   
                             2024-06-30             2148.00           169.2   

                                           Comprimento médio (mm)  Diâmetro  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03                     110.4      32.5   
                             2024-01-04                      85.6      18.1   
                             2024-01-05                     140.5      32.9   
                             2024-01-06                     144.7      32.7   
                             2024-01-07                     141.5      30.2   
...                                                           ...       ...   
194121        C0516.0000R    2024-06-26                     124.3      32.6   
                             2024-06-27                     120.2      33.6   
                             2024-06-28                     124.1      34.2   
                             2024-06-29                     120.0      33.5   
                             2024-06-30                     122.9      34.6   

                                           Total de Defeitos no Dia  \
Ordem de prod Ref. do Artigo Data de prod                             
192970        C0516.0000R    2024-01-03                         222   
                             2024-01-04                         249   
                             2024-01-05                         268   
                             2024-01-06                         242   
                             2024-01-07                         232   
...                                                             ...   
194121        C0516.0000R    2024-06-26                         258   
                             2024-06-27                         218   
                             2024-06-28                         218   
                             2024-06-29                         213   
                             2024-06-30                         256   

                                          Defeito Mais Frequente do Dia  \
Ordem de prod Ref. do Artigo Data de prod                                 
192970        C0516.0000R    2024-01-03                    Sujo de óleo   
                             2024-01-04                    Sujo de óleo   
                             2024-01-05                    Sujo de óleo   
                             2024-01-06                    Sujo de óleo   
                             2024-01-07                    Sujo de óleo   
...                                                                 ...   
194121        C0516.0000R    2024-06-26                    Sujo de óleo   
                             2024-06-27                    Sujo de óleo   
                             2024-06-28                    Sujo de óleo   
                             2024-06-29                    Sujo de óleo   
                             2024-06-30                    Sujo de óleo   

                                           Sujo de óleo  Rugas  \
Ordem de prod Ref. do

### Verificando se o dataframe está limpo e padronizado

In [517]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 200 entries, (192970, 'C0516.0000R', '2024-01-03') to (194121, 'C0516.0000R', '2024-06-30')
Columns: 133 entries, Temperatura (F°) to Crú
dtypes: float64(4), int64(128), object(1)
memory usage: 216.2+ KB


In [518]:
df.isnull().sum()

Temperatura (F°)            0
Peso médio (g)              0
Comprimento médio (mm)      0
Diâmetro                    0
Total de Defeitos no Dia    0
                           ..
Ferrugem no corpo           0
Fagulha                     0
Alça deformada              0
Instabilidade               0
Crú                         0
Length: 133, dtype: int64

In [519]:
df_reset = df.reset_index()

In [520]:
df_reset

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Total de Defeitos no Dia,Defeito Mais Frequente do Dia,Sujo de óleo,...,Bolha aberta interna,Cascão cortante,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú
0,192970,C0516.0000R,2024-01-03,2126.97,169.3,110.4,32.5,222,Sujo de óleo,31,...,0,0,0,0,0,0,0,0,0,0
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,249,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
2,192970,C0516.0000R,2024-01-05,2125.00,173.6,140.5,32.9,268,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
3,192970,C0516.0000R,2024-01-06,2125.40,170.2,144.7,32.7,242,Sujo de óleo,73,...,0,0,0,0,0,0,0,0,0,0
4,192970,C0516.0000R,2024-01-07,2126.76,171.7,141.5,30.2,232,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,194121,C0516.0000R,2024-06-26,2154.44,169.0,124.3,32.6,258,Sujo de óleo,40,...,1,0,0,0,0,0,0,0,0,0
196,194121,C0516.0000R,2024-06-27,2152.32,168.9,120.2,33.6,218,Sujo de óleo,75,...,0,0,0,0,0,0,0,0,0,0
197,194121,C0516.0000R,2024-06-28,2145.60,169.3,124.1,34.2,218,Sujo de óleo,54,...,0,0,0,0,0,0,0,0,0,0
198,194121,C0516.0000R,2024-06-29,2145.60,168.8,120.0,33.5,213,Sujo de óleo,54,...,0,0,0,0,0,0,1,0,0,0


# Solução - Prever total de defeitos e defeitos por dia

## 1. Complementar a previsão de defeitos
O modelo pode ser alimentado não apenas pelas características do pedido (medidas), mas também pela duração da ordem para calcular mais precisamente o total de defeitos.
Depoi de analisar, ordens mais longas tendem a ter mais defeitos. Portanto, a duração poderia atuar como um fator adicional importante, ajudando o modelo a ajustar sua previsão de defeitos com base na duração real da ordem.

## 2. Prever o total de defeitos, defeitos por dia e dia crítico 
Com a duração da ordem fornecida, podemos ajustar a sazonalidade dos defeitos. A ideia seria usar a duração para:

Mapear quais dias da ordem têm maior probabilidade de defeitos, mesmo antes da produção começar. Ou seja, o modelo pode indicar se a ordem tende a ter mais defeitos nos primeiros dias ou se a distribuição dos defeitos será mais uniforme ao longo da produção.
Com ordens de duração maior, a previsão de defeitos poderia ser ajustada para mostrar que mais defeitos podem ocorrer nos dias intermediários ou finais.

### Padrões Cíclicos: 
Se, por exemplo, o Dia 2 costuma ter um número elevado de defeitos, isso pode sugerir problemas recorrentes ou características intrínsecas do processo nesse estágio.

### Comparação Escalável: 
Como as ordens têm durações diferentes, é difícil compará-las diretamente. Normalizar os dias cria um padrão comum que facilita a análise.

### Previsibilidade: 
A partir dessa análise, você pode prever que, em uma nova ordem, o Dia 2 (ou qualquer outro dia normalizado) tem maior probabilidade de apresentar mais defeitos.

## 3. Modelagem ajustada pela duração
Criar uma lógica que leva em consideração a duração da ordem:

**Entradas**: Receberia não apenas as medidas físicas (peso, comprimento, diâmetro) e a duração da ordem, mas também a data de início ou a quantidade de dias de produção.

**Processamento**: Com a duração, o modelo poderia ajustar a previsão de defeitos, utilizando o padrão de sazonalidade (dias de maior probabilidade de defeitos).

**Saída**: O modelo retornaria tanto o total de defeitos estimados quanto os dias críticos de defeitos (e.g., o segundo dia da ordem, conforme você observou, com maior chance de defeitos).


In [521]:
df_sazonalidade = df_reset.groupby(['Ordem de prod', 'Data de prod'])[['Total de Defeitos no Dia']].sum().reset_index()

In [522]:
df_sazonalidade['Data de prod'] = pd.to_datetime(df_sazonalidade['Data de prod'])

In [523]:
df_sazonalidade['Dia da Semana'] = df_sazonalidade['Data de prod'].dt.day_name()
df_sazonalidade['Mês'] = df_sazonalidade['Data de prod'].dt.month
df_sazonalidade['Dia do Ano'] = df_sazonalidade['Data de prod'].dt.dayofyear

In [524]:
duracao_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Data de prod'].nunique().reset_index()
duracao_por_ordem.columns = ['Ordem de prod', 'Dias de Duração']

duracao_por_ordem

,Ordem de prod,Dias de Duração
0,192970,9
1,193112,3
2,193164,12
3,193206,4
4,193207,10
5,193208,2
6,193209,7
7,193342,20
8,193368,7
9,193459,3


In [525]:
defeitos_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Total de Defeitos no Dia'].sum().reset_index()
defeitos_por_ordem.columns = ['Ordem de prod', 'Total de Defeitos por Ordem']

In [526]:
duracao_por_ordem = duracao_por_ordem.merge(defeitos_por_ordem, on='Ordem de prod', how='left')

In [527]:
px.scatter(data_frame=duracao_por_ordem, 
           x='Dias de Duração', 
           y='Total de Defeitos por Ordem', 
           color='Total de Defeitos por Ordem', 
           color_continuous_scale='peach', 
           size='Total de Defeitos por Ordem')

In [528]:
df_sazonalidade = df_sazonalidade.merge(duracao_por_ordem, on='Ordem de prod', how='left')

In [529]:
df_sazonalidade

,Ordem de prod,Data de prod,Total de Defeitos no Dia,Dia da Semana,Mês,Dia do Ano,Dias de Duração,Total de Defeitos por Ordem
0,192970,2024-01-03,222,Wednesday,1,3,9,2148
1,192970,2024-01-04,249,Thursday,1,4,9,2148
2,192970,2024-01-05,268,Friday,1,5,9,2148
3,192970,2024-01-06,242,Saturday,1,6,9,2148
4,192970,2024-01-07,232,Sunday,1,7,9,2148
...,...,...,...,...,...,...,...,...
195,194121,2024-06-26,258,Wednesday,6,178,6,1436
196,194121,2024-06-27,218,Thursday,6,179,6,1436
197,194121,2024-06-28,218,Friday,6,180,6,1436
198,194121,2024-06-29,213,Saturday,6,181,6,1436


### Criando Dataframe agrupado por ordem de produção, com operações diferentes

In [530]:
defeitos_por_duracao = df_sazonalidade.groupby('Ordem de prod').agg({'Total de Defeitos no Dia': 'sum', 'Dias de Duração': 'first'})

### Renomear a coluna 'Total de Defeitos no Dia' para 'Total de Defeitos por Ordem'

In [531]:
defeitos_por_duracao.rename(columns={'Total de Defeitos no Dia': 'Total de Defeitos por Ordem'}, inplace=True)

In [532]:
defeitos_por_duracao

,Total de Defeitos por Ordem,Dias de Duração
Ordem de prod,,
192970,2148,9
193112,584,3
193164,2661,12
193206,1041,4
193207,2130,10
193208,369,2
193209,1807,7
193342,3685,20
193368,1464,7


## Média de defeitos por dias da semana

In [533]:
df_medias_dia_semana = df_sazonalidade.groupby('Dia da Semana')[['Total de Defeitos no Dia']].mean().sort_values('Total de Defeitos no Dia', ascending=False).round(2)

In [534]:
fig = px.bar(df_medias_dia_semana.reset_index(), 
             x='Dia da Semana', y='Total de Defeitos no Dia',
             text='Total de Defeitos no Dia')

fig.update_traces(textposition='outside', texttemplate='%{text}', 
                  textfont_size=14, textfont_color='black')

fig.show()

## Média de defeitos por dia de produção

### Calcular a duração total de cada ordem

In [535]:
duracao_por_ordem = df_sazonalidade.groupby('Ordem de prod')['Data de prod'].nunique().reset_index()
duracao_por_ordem.columns = ['Ordem de prod', 'Dias de Duração']

### Merge com o DataFrame original

In [536]:
df_sazonalidade = df_sazonalidade.merge(duracao_por_ordem, on='Ordem de prod', how='left')

### Remover a coluna duplicada (se já havia 'Dias de Duração' no DataFrame original)

In [537]:
df_sazonalidade = df_sazonalidade.drop(columns=['Dias de Duração_x'])

### Renomear a coluna restante, se necessário

In [538]:
df_sazonalidade = df_sazonalidade.rename(columns={'Dias de Duração_y': 'Dias de Duração'})

### Criar a coluna 'Dia Normalizado'

Calcular o Dia Contado por Ordem

In [539]:
df_sazonalidade['Dia Contado'] = df_sazonalidade.groupby('Ordem de prod').cumcount() + 1

Calcular o Dia Normalizado

In [540]:
df_sazonalidade['Dia Normalizado'] = df_sazonalidade['Dia Contado'] / df_sazonalidade['Dias de Duração']

Visualização

In [541]:
df_sazonalidade[['Ordem de prod', 'Dia Contado', 'Dias de Duração', 'Dia Normalizado']]

,Ordem de prod,Dia Contado,Dias de Duração,Dia Normalizado
0,192970,1,9,0.111111
1,192970,2,9,0.222222
2,192970,3,9,0.333333
3,192970,4,9,0.444444
4,192970,5,9,0.555556
...,...,...,...,...
195,194121,2,6,0.333333
196,194121,3,6,0.500000
197,194121,4,6,0.666667
198,194121,5,6,0.833333


### Calcular a média de defeitos por Dia Normalizado

In [542]:
defeitos_por_dia_normalizado = df_sazonalidade.groupby('Dia Normalizado')[['Total de Defeitos no Dia']].mean().reset_index()
defeitos_por_dia_normalizado.columns = ['Dia Normalizado', 'Média de Defeitos']

Visualização

In [585]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=defeitos_por_dia_normalizado['Dia Normalizado'],
                         y=defeitos_por_dia_normalizado['Média de Defeitos'],
                         mode='lines+markers', 
                         marker=dict(size=10),
                         line=dict(color='red'),
                         name='Média de Defeitos'))

fig.update_layout(
    title='Média de Defeitos por Dia Normalizado',
    title_font_size=16,
    xaxis_title='Dia Normalizado',
    xaxis_title_font_size=14,
    yaxis_title='Média de Defeitos',
    yaxis_title_font_size=14
)

fig.show()

### Calcular a média de defeitos por Dia Contado

Agrupar pelos valores inteiros de Dia Contado

In [544]:
defeitos_por_dia_contado = df_sazonalidade.groupby('Dia Contado')[['Total de Defeitos no Dia']].mean().reset_index()
defeitos_por_dia_contado.columns = ['Dia Contado', 'Média de Defeitos']

Visualização

In [593]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=defeitos_por_dia_contado['Dia Contado'],
                         y=defeitos_por_dia_contado['Média de Defeitos'],
                         mode='lines+markers', 
                         marker=dict(size=10),
                         line=dict(color='red'),
                         name='Média de Defeitos'))

fig.update_layout(
    title='Média de Defeitos por Dia Contado',
    title_font_size=16,
    xaxis_title='Dia Contado',
    xaxis_title_font_size=14,
    yaxis_title='Média de Defeitos',
    yaxis_title_font_size=14
)

fig.show()

## Implementação do Modelo de Regressão

### Resetando index

In [546]:
df_total_duracao_reset = defeitos_por_duracao.reset_index()

In [547]:
df_total_duracao_reset

,Ordem de prod,Total de Defeitos por Ordem,Dias de Duração
0,192970,2148,9
1,193112,584,3
2,193164,2661,12
3,193206,1041,4
4,193207,2130,10
5,193208,369,2
6,193209,1807,7
7,193342,3685,20
8,193368,1464,7
9,193459,505,3


### Passo 1: Preparar os dados para o total de defeitos e a duração da ordem

In [548]:
df_combined = df_reset.merge(df_total_duracao_reset[['Ordem de prod', 'Total de Defeitos por Ordem', 'Dias de Duração']], on='Ordem de prod', how='left')

### Passo 2: Separar variáveis independentes (medidas e duração) e dependente (total de defeitos)

In [549]:
X = df_combined[['Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro', 'Dias de Duração']]
y = df_combined['Total de Defeitos por Ordem']

### Passo 3: Dividir os dados em treino e teste

In [550]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Passo 4: Normalizar os dados

In [551]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Passo 5: Treinar o modelo para prever o total de defeitos

In [552]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_scaled, y_train)

GradientBoostingRegressor(random_state=42)

### Passo 6: Previsões de total de defeitos

In [553]:
y_pred_gb = gb_model.predict(X_test_scaled)

### Passo 7: Avaliar a performance do modelo

In [554]:
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f"Gradient Boosting - Mean Squared Error (MSE): {mse_gb}")
print(f"Gradient Boosting - R² Score: {r2_gb}")

Gradient Boosting - Mean Squared Error (MSE): 7698.757546270441
Gradient Boosting - R² Score: 0.9918282458806885


## Testando modelo

### Criando nova ordem e prevendo com o modelo

In [555]:
nova_ordem = {
    'Peso médio (g)': 160,
    'Comprimento médio (mm)': 29,
    'Diâmetro': 15.9,
    'Dias de Duração': 9
}

X_nova_ordem = scaler.transform(pd.DataFrame([nova_ordem])) 
total_defeitos_previsto = gb_model.predict(X_nova_ordem)

### Calcular a soma total dos defeitos para garantir que a soma seja igual ao total esperado

In [556]:
total_medio_defeitos = defeitos_por_dia_contado['Média de Defeitos'].sum()

### Garantir que a soma dos defeitos seja proporcionalmente ajustada ao total de defeitos previsto

In [557]:
defeitos_por_dia_contado['Defeitos Previstos'] = (defeitos_por_dia_contado['Média de Defeitos'] / total_medio_defeitos) * total_defeitos_previsto

### Recalcular a distribuição dos defeitos considerando o número de dias de duração

In [558]:
dias_duracao = nova_ordem['Dias de Duração']

### Selecionar as previsões apenas para os dias da nova ordem

In [559]:
df_dias_duracao = defeitos_por_dia_contado.head(dias_duracao).copy()  # Usando .copy() para evitar aviso

### Garantir que a soma dos defeitos nos dias da nova ordem seja igual ao total de defeitos previsto


Ajustar a distribuição proporcionalmente para que a soma dos defeitos seja igual ao total_defeitos_previsto

In [560]:
soma_defeitos_dia = df_dias_duracao['Defeitos Previstos'].sum()
ajuste_fator = total_defeitos_previsto / soma_defeitos_dia

# Ajustar os defeitos previstos diretamente no DataFrame com .loc[]
df_dias_duracao.loc[:, 'Defeitos Previstos'] = df_dias_duracao['Defeitos Previstos'] * ajuste_fator

### Visualização Geral

In [587]:
print(f"Total de defeitos previsto para a nova ordem: {total_defeitos_previsto[0]}")

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_dias_duracao['Dia Contado'],
                         y=df_dias_duracao['Defeitos Previstos'],
                         mode='lines+markers',
                         marker=dict(size=10),
                         line=dict(color='green'),
                         name='Defeitos Previstos'))

fig.update_layout(
    title=f'Previsão de Defeitos ao Longo dos {dias_duracao} Dias da Nova Ordem',
    title_font_size=16,
    xaxis_title='Dia de Produção',
    xaxis_title_font_size=14,
    yaxis_title='Defeitos Previstos',
    yaxis_title_font_size=14
)

fig.show()

df_dias_duracao[['Dia Contado', 'Defeitos Previstos']].set_index('Dia Contado')

Total de defeitos previsto para a nova ordem: 2130.1742127476737


,Defeitos Previstos
Dia Contado,
1,245.596208
2,270.117364
3,236.513327
4,242.914374
5,236.915285
6,234.991332
7,206.332975
8,221.010944
9,235.782403


# Analisando 2º dia de produção das ordens

In [562]:
# Fazer o merge entre o df_reset e o duracao_por_ordem com base na 'Ordem de prod'
df_reset_merged = df_reset.merge(duracao_por_ordem, on='Ordem de prod', how='left')

# Garantir que 'Data de prod' seja do tipo datetime
df_reset_merged['Data de prod'] = pd.to_datetime(df_reset_merged['Data de prod'])

# Agrupar por 'Ordem de prod' e calcular o 'Dia Contado' para cada grupo
df_reset_merged['Dia Contado'] = df_reset_merged.groupby('Ordem de prod')['Data de prod'].transform(lambda x: (x - x.min()).dt.days + 1)

# Verificar o resultado
df_reset_merged

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Total de Defeitos no Dia,Defeito Mais Frequente do Dia,Sujo de óleo,...,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú,Dias de Duração,Dia Contado
0,192970,C0516.0000R,2024-01-03,2126.97,169.3,110.4,32.5,222,Sujo de óleo,31,...,0,0,0,0,0,0,0,0,9,1
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,249,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,9,2
2,192970,C0516.0000R,2024-01-05,2125.00,173.6,140.5,32.9,268,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,9,3
3,192970,C0516.0000R,2024-01-06,2125.40,170.2,144.7,32.7,242,Sujo de óleo,73,...,0,0,0,0,0,0,0,0,9,4
4,192970,C0516.0000R,2024-01-07,2126.76,171.7,141.5,30.2,232,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,194121,C0516.0000R,2024-06-26,2154.44,169.0,124.3,32.6,258,Sujo de óleo,40,...,0,0,0,0,0,0,0,0,6,2
196,194121,C0516.0000R,2024-06-27,2152.32,168.9,120.2,33.6,218,Sujo de óleo,75,...,0,0,0,0,0,0,0,0,6,3
197,194121,C0516.0000R,2024-06-28,2145.60,169.3,124.1,34.2,218,Sujo de óleo,54,...,0,0,0,0,0,0,0,0,6,4
198,194121,C0516.0000R,2024-06-29,2145.60,168.8,120.0,33.5,213,Sujo de óleo,54,...,0,0,0,0,1,0,0,0,6,5


In [564]:
dia_contado_agrupado = df_reset_merged.groupby('Dia Contado')[['Temperatura (F°)', 'Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro']].mean()

In [565]:
dia_contado_agrupado_reset = dia_contado_agrupado.reset_index()

px.line(data_frame=dia_contado_agrupado_reset, x='Dia Contado', y='Comprimento médio (mm)')

In [566]:
ordens_dia_2 = df_reset_merged.loc[df_reset_merged['Dia Contado' ] == 2, :]

In [567]:
ordens_dia_2

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Total de Defeitos no Dia,Defeito Mais Frequente do Dia,Sujo de óleo,...,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú,Dias de Duração,Dia Contado
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,249,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,9,2
10,193112,C1164.0000R,2024-01-02,2189.08,168.3,77.6,41.6,203,Sujo de óleo,125,...,0,0,0,0,0,0,0,0,3,2
13,193164,C1280.0000R,2024-01-12,2128.92,148.5,95.2,35.7,240,Sujo de óleo,98,...,0,0,0,0,0,0,0,0,12,2
25,193206,C1284.0000R,2024-01-26,2177.37,171.2,91.9,38.6,255,Sujo de óleo,50,...,0,0,0,0,0,0,0,0,4,2
29,193207,C1298.0000R,2024-01-29,2144.11,170.6,99.8,37.1,243,Falta de seção,63,...,0,0,0,0,0,0,0,0,10,2
39,193208,C0104.0000R,2024-02-08,2171.88,122.2,46.7,20.7,242,Fora de cor,29,...,0,0,0,0,0,0,0,0,2,2
41,193209,C0509.0000R,2024-02-09,2128.06,191.0,87.3,18.6,305,Sujo de óleo,65,...,0,0,0,0,0,0,0,0,7,2
48,193342,C0516.0000R,2024-02-22,2126.32,171.2,107.6,28.1,296,Sujo de óleo,51,...,0,0,0,0,0,0,0,0,20,2
75,193459,C0068.0000R,2024-03-12,2147.79,194.2,95.9,40.1,189,Sujo de óleo,62,...,0,0,0,0,0,0,0,0,3,2
78,193462,C1284.0000R,2024-03-22,2173.72,172.6,86.0,40.4,265,Trinca por choque térmico,39,...,0,0,0,0,0,0,0,0,5,2
